In [1]:
from agents import Agent, Runner, set_default_openai_key
from agents.mcp import MCPServerStdio
import os 
import json

# If the API keys are not already in your environment, you can set them here in <your_atla_api_key> and <your_openai_api_key>
atla_api_key = os.environ.get("ATLA_API_KEY", "<your_atla_api_key>")
openai_api_key = os.environ.get("OPENAI_API_KEY", "<your_openai_api_key>")
set_default_openai_key(openai_api_key)

In [2]:
async def run_agent_with_atla(user_request):
    async with MCPServerStdio(
            params={
                "command": "npx",
                "args": [
                    "mcp-remote",
                    "http://localhost:8001/sse",
                    "--header",
                    f"Authorization: Bearer {atla_api_key}"
                ],
                "env": {"ATLA_API_KEY":atla_api_key}
            }
        ) as atla_mcp_server:
            # Create an agent with the Atla evaluation server
            agent = Agent(
                name="AssistantWithAtlaEval",
                instructions="""
                You are a helpful assistant. Your goal is to provide high-quality responses to user requests.
                You can use the Atla evaluation tool to improve your responses.
                """,
                mcp_servers=[atla_mcp_server], # You can equip any Agent with Atla's MCP server like this
                model="gpt-4o-mini"
            )
            
            # Run the agent with the user request
            result = await Runner.run(agent, user_request)
            
            # Print the MCP call result
            for item in result.new_items:
                if item.type == "message_output_item":
                    print("\n*******  Agent's response: ******")
                    print(item.raw_item.content[0].text)
                elif item.type == 'tool_call_item':
                    print("\n*******  Agent's request to Atla tool: ******:")
                    print(json.loads(item.raw_item.arguments))
                elif item.type == 'tool_call_output_item':
                    print("\n*******  Atla's response to Agent's request: ******:")
                    print(json.loads(item.output)['text'])

In [3]:
await run_agent_with_atla(user_request="Write a one-line poem about the ocean. Evaluate it with atla for cliche and improve it once using the feedback.")


*******  Agent's request to Atla tool: ******:
{'model_input': 'Write a one-line poem about the ocean.', 'model_output': 'The ocean whispers secrets to the shore, ancient tales that echo forevermore.', 'evaluation_criteria': 'Evaluate the poem for cliche and originality.', 'expected_model_output': 'A more original line could replace common phrases and imagery with fresh insights.'}

*******  Atla's response to Agent's request: ******:
{"score": "4", "critique": "The poem effectively captures the essence of the ocean, but it relies on familiar imagery and phrasing, such as \"whispers secrets\" and \"echo forevermore,\" which, while evocative, are somewhat clich\u00e9d. A more original line could replace common phrases and imagery with fresh insights, offering a unique perspective on the ocean's mystique. To enhance originality, consider incorporating sensory details or metaphors that are less commonly associated with the ocean, thereby adding depth and surprise to the poem."}

******* 